10. Write a program to predict the winning team in IPL matches

In [ ]:
#!pip install pandas==1.5.3

In [ ]:
#importing the csv file
import pandas as pd
iplmatches=pd.read_csv("matches.csv")
iplmatches

In [ ]:
iplmatches.info()
#information about iplmatches

In [ ]:
iplmatches.describe()
#Statistics about ipl matches

In [ ]:
iplmatches.columns
#Checking the columns

In [ ]:
iplmatches.isnull().sum()

In [ ]:

iplmatches.drop(['umpire3'],axis=1,inplace=True)


In [ ]:
iplmatches.columns

In [ ]:
teams_per_season = iplmatches.groupby('season')['winner'].value_counts()
teams_per_season

In [ ]:
year = 2008
winteamseason = pd.DataFrame(columns=['year', 'team', 'wins'])
winteamseason

In [ ]:
for items in teams_per_season.iteritems():
    if items[0][0]==year:
        print(items)
        win_series = pd.DataFrame({
            'year': [items[0][0]],
            'team': [items[0][1]],
            'wins': [items[1]]
        })
        winteamseason = winteamseason.append(win_series)
        year += 1

print(winteamseason)

In [ ]:
winteamseason.describe()

In [ ]:
venue = iplmatches['venue'].value_counts()
print(venue)

In [ ]:
venue_df = pd.DataFrame(columns=['venue', 'matches'])
for items in venue.iteritems():
    temp_df = pd.DataFrame({
        'venue':[items[0]],
        'matches':[items[1]]
    })
    venue_df = venue_df.append(temp_df, ignore_index=True)

In [ ]:
team_wins_ser = iplmatches['winner'].value_counts()

team_wins_df = pd.DataFrame(columns=["team", "wins"])
for items in team_wins_ser.iteritems():
    temp_df1 = pd.DataFrame({
        'team':[items[0]],
        'wins':[items[1]]
    })
    team_wins_df = team_wins_df.append(temp_df1, ignore_index=True)

In [ ]:
mvp_ser = iplmatches['player_of_match'].value_counts()

mvp_ten_df = pd.DataFrame(columns=["player", "wins"])
count = 0
for items in mvp_ser.iteritems():
    if count>9:
        break
    else:
        temp_df2 = pd.DataFrame({
            'player':[items[0]],
            'wins':[items[1]]
        })
        mvp_ten_df = mvp_ten_df.append(temp_df2, ignore_index=True)
        count += 1

In [ ]:
toss_ser = iplmatches['toss_winner'].value_counts()

toss_df = pd.DataFrame(columns=["team", "wins"])

for items in toss_ser.iteritems():
    temp_df3 = pd.DataFrame({
        'team':[items[0]],
        'wins':[items[1]]
    })
    toss_df = toss_df.append(temp_df3, ignore_index=True)

In [ ]:
iplmatches.loc[iplmatches["winner"]==iplmatches["team1"],"team1_win"]=1
iplmatches.loc[iplmatches["winner"]!=iplmatches["team1"],"team1_win"]=0

#outcome variable team1_toss_win as a value of team1 winning the toss
iplmatches.loc[iplmatches["toss_winner"]==iplmatches["team1"],"team1_toss_win"]=1
iplmatches.loc[iplmatches["toss_winner"]!=iplmatches["team1"],"team1_toss_win"]=0

#outcome variable team1_bat to depict if team1 bats first
iplmatches["team1_bat"]=0
iplmatches.loc[(iplmatches["team1_toss_win"]==1) & (iplmatches["toss_decision"]=="bat"),"team1_bat"]=1

In [ ]:
prediction_df=iplmatches[["team1","team2","team1_toss_win","team1_bat","team1_win","venue"]]

#finding the higly correlated features
correlated_features = set()
correlation_matrix = prediction_df.drop('team1_win', axis=1).corr()

for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.9:
            column = correlation_matrix.columns[i]
            correlated_features.add(column)

prediction_df.drop(columns=correlated_features)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

X = prediction_df.drop('team1_win', axis=1)
target = prediction_df['team1_win']
target=target.astype(int)
X=pd.get_dummies(X)

X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state=0,shuffle=True)

In [ ]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

In [ ]:
svm=SVC()
svm.fit(X_train,y_train)


In [ ]:
y_pred=svm.predict(X_test)
y_pred
X_test['actual']=y_test
X_test['pred']=y_pred

print(X_test.columns)
print(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
